In [ ]:
import pandas as pd
from mistralai import Mistral
import time

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")


df = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets.csv")

# Fonction pour analyser le sentiment d'un tweet avec un délai
def get_sentiment(tweet):
    content = "Analyse du sentiment du tweet : " + tweet
    response = client.agents.complete(
        agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    # Récupérer la valeur du sentiment dans la réponse JSON
    response_text = response.choices[0].message.content.strip()
    return response_text

# Appliquer la fonction à chaque tweet avec un délai de 4 secondes entre chaque requête
sentiments = []
for tweet in df['tweet_text']:
    sentiment = get_sentiment(tweet)
    sentiments.append(sentiment)
    time.sleep(4)  # Pause de 4 secondes entre les requêtes

# Ajouter les résultats dans la nouvelle colonne "sentiment"
df['sentiment'] = sentiments

# Afficher le DataFrame avec la nouvelle colonne "sentiment"
df.head()


KeyboardInterrupt: 

In [33]:
from mistralai import Mistral

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")

# Le contenu du tweet que tu veux analyser
content = "C'est une super journée, je suis tellement content !"

# Ajout d'un préfixe pour spécifier l'analyse de sentiment
content = "Analyse du sentiment du tweet : " + content

# Utilisation de l'agent pour obtenir la réponse
response = client.agents.complete(
    agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
    messages=[{
        "role": "user",
        "content": content
    }]
)

# Récupérer et afficher la réponse générée par l'agent
response_text = response.choices[0].message.content.strip()
print("Sentiment analysé : ", response_text)


Sentiment analysé :  {
  "inconfort": 0,
  "sentiment": positif,
  "urgence": False
}


In [43]:
import pandas as pd
import json
import re

# Charger le DataFrame
df = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets_with_sentiment.csv")

# Fonction pour extraire la valeur de la clé "Sentiment"
def extract_sentiment(json_str):
    try:
        # Isoler la partie JSON avant "Explication" ou autre texte ajouté
        json_part = re.search(r'\{.*?\}', json_str, re.S).group(0)
        
        # Nettoyer et parser le JSON
        cleaned_str = json_part.replace('\r\n', '').replace("'", '"')
        json_dict = json.loads(cleaned_str)

        # Retourner la valeur de la clé "Sentiment"
        return json_dict.get("Sentiment")
    except (json.JSONDecodeError, TypeError, AttributeError):
        return None

# Appliquer la fonction pour extraire les sentiments
df['sentiment'] = df['sentiment'].apply(extract_sentiment)

# Fonction pour séparer la date et l'heure
def split_date_time(datetime_str):
    try:
        date_part, time_part = datetime_str.split(' ')
        time_part = time_part.replace('+00:00', '')  # Supprimer le décalage horaire
        return date_part, time_part
    except Exception:
        return None, None

# Appliquer la séparation sur la colonne 'date'
df['date'], df['heure'] = zip(*df['date'].apply(split_date_time))

# Convertir les colonnes au bon format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convertir en datetime
df['heure'] = pd.to_datetime(df['heure'], format='%H:%M:%S', errors='coerce').dt.time  # Convertir en time

# ✅ Convertir 'user_id' en chaîne de caractères et enlever les zéros inutiles
df['user_id'] = df['user_id'].apply(lambda x: str(int(x)) if pd.notna(x) else None)


# Vérifier les résultats
df.head()




,user_id,screen_name,name,date,tweet_text,hour,tweet_length,criticite,sentiment,heure
0,189684000000000011534336,lescausesjustes,AInnovation,2025-03-04,"@ENGIEgroup @ENGIEpartFR pour l'amour du ciel,...",8,113,1,Négatif,08:09:58
1,18967000000000000262144,Leoletonneau,Tigre & Dragon 🐯 🐉,2025-03-03,"@ENGIEpartFR vraiment des escrocs, des montant...",23,131,1,Négatif,23:25:45
2,189662999999999998164992,SouareFirst,So First,2025-03-03,@ENGIEpartFR bonjour engie les voleurs ! Atten...,18,184,1,Négatif,18:47:06
3,189661999999999987941376,julien_ducerf,Julien Ducerf,2025-03-03,@ENGIEpartFR c'est du harcèlement pour augment...,17,167,2,Négatif,17:50:42
4,189658000000000014155776,djofthp,jojodamido,2025-03-03,@ENGIEpartFR on vous parle,15,26,1,Neutre,15:24:10


In [56]:
import pandas as pd
import time
from mistralai import Mistral

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")

# Filtrer les tweets négatifs
df_negatifs = df[df['sentiment'] == 'Négatif'].copy()

# Fonction pour analyser chaque tweet avec gestion des erreurs
def analyse_tweet_with_retry(tweet, retries=3, delay=5):
    attempt = 0
    while attempt < retries:
        try:
            response = client.agents.complete(
                agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
                messages=[{
                    "role": "user",
                    "content": tweet
                }]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "429" in str(e):  # Si l'erreur est un dépassement de quota
                print(f"Rate limit exceeded, waiting for {delay} seconds...")
                time.sleep(delay)  # Attente avant de réessayer
                attempt += 1
            else:
                print(f"Une erreur est survenue: {e}")
                raise  # Si l'erreur est autre, relancer l'exception
    return None  # Si toutes les tentatives échouent

# Appliquer la fonction d'analyse avec un délai et gestion des erreurs sur chaque tweet du DataFrame
def apply_analysis_with_delay_and_retry(row):
    result = analyse_tweet_with_retry(row['tweet_text'])
    # Attendre quelques secondes entre les requêtes (par exemple, 5 secondes)
    time.sleep(5)  # Ajuste la durée en fonction de la limite de l'API
    return result

# Appliquer la fonction d'analyse avec un délai sur les tweets négatifs
df_negatifs['sentiment_analysis'] = df_negatifs.apply(apply_analysis_with_delay_and_retry, axis=1)

# Fusionner les résultats dans le DataFrame original `df` en ajoutant la colonne `sentiment_analysis`
df = df.merge(df_negatifs[['tweet_text', 'sentiment_analysis']], on='tweet_text', how='left')


In [58]:
df.to_csv("data_tweet_ready.csv", index=False)  # Remplace par ton chemin de sauvegarde